In [73]:
!pip freeze | grep scikit-learn

scikit-learn @ file:///tmp/build/80754af9/scikit-learn_1642617106979/work
scikit-learn-intelex==2021.20220215.212715


In [74]:
import pickle
import pandas as pd
from datetime import datetime

In [75]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [76]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [77]:
# df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')
df = read_data('data/fhv_tripdata_2021-02.parquet')

In [78]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

**Q1. Mean predicted duration for this dataset**

In [79]:
y_pred.mean()

16.191691679979066

**Q2. Preparing the output**

In [85]:
df['ride_id'] = df['pickup_datetime'].apply(lambda x: f"{pd.to_datetime(x).year:04d}/{pd.to_datetime(x).month:02d}_") + df.index.astype('str')


In [90]:
df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predictions'] = y_pred
df_result

,ride_id,predictions
1,2021/02_1,14.539865
2,2021/02_2,13.740422
3,2021/02_3,15.593339
4,2021/02_4,15.188118
5,2021/02_5,13.817206
...,...,...
1037687,2021/02_1037687,12.433246
1037688,2021/02_1037688,12.910885
1037689,2021/02_1037689,15.834923
1037690,2021/02_1037690,16.783176


In [91]:
output_file = 'data/fhv_tripdata_2021-02_test.parquet'
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)